# In Pydantic



# # LangGraph's State Update Mechanism:

In LangGraph node functions, you must return dictionaries regardless of whether you use TypedDict or Pydantic for the State definition. This is how LangGraph's state update mechanism works.


Node receives state (as Pydantic object or dict)
Node returns dictionary with updates
LangGraph merges the returned dict with existing state
add_messages function handles message list merging

The Rule:

Graph Input: Can be State object OR dictionary
Node Functions: MUST return dictionaries
Graph Output: Always dictionary

```python
# 1. Input to graph (can be object or dict)
initial_state = State(messages=[...])  # OR {"messages": [...]}

# 2. Inside node1
def node1(state: State):  # Receives State object
    return {"messages": [...]}  # MUST return dict

# 3. LangGraph merges the dict with existing state
# 4. Next node receives updated state
def tool_calling_llm(state: State):  # Receives updated State
    return {"messages": [...]}  # MUST return dict

# 5. Final output from graph
result = graph.invoke(initial_state)  # Always returns dict
```

So you are using the simple dictionary approach where it's required (in node return values) - this is the only way LangGraph node functions can work!


## partial Updates

```python
# Update only messages, leave other fields unchanged
def update_messages_only(state: State):
    return {"messages": [new_message]}
    # tool_input and tool_output remain unchanged!

# Update only tool_input
def update_tool_only(state: State):
    return {"tool_input": "new value"}
    # messages and tool_output remain unchanged!

# Update multiple fields
def update_multiple(state: State):
    return {
        "messages": [new_message],
        "tool_input": "updated"
    }
    # tool_output remains unchanged!
```




Node functions must return dictionaries because:

LangGraph's architecture expects dictionary keys to know what to update
Reducer functions like add_messages need to be called on specific fields
Partial updates are only possible with dictionary keys
Type safety is maintained while allowing flexible updates
Framework design - this is how LangGraph fundamentally works

It's not about preference - it's about how LangGraph's state management system is architected!


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from typing import Annotated, List
from pydantic import BaseModel, Field
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

import os, getpass
from pprint import pprint

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

def llm():
    return ChatOpenAI(model="gpt-4o")

# ✅ Use Pydantic BaseModel
class State(BaseModel):
    messages: Annotated[List[BaseMessage], add_messages] = Field(default_factory=list)
    tool_input: str = Field(default="")
    tool_output: str = Field(default="")

    class Config:
        arbitrary_types_allowed = True  # Allow BaseMessage types

@tool
def multiply(x: int, y: int) -> int:
    """Multiply two integers.

    Args:
        x: First integer
        y: Second integer
    """
    return x * y

def node1(state: State):
    """Initialize conversation if no messages exist."""
    # Check if messages already exist
    return {
            "messages": [HumanMessage(content="What is 4 times 4")],
            "tool_input": "",
            "tool_output": "",
    }


def tool_calling_llm(state: State):
    """LLM node that can call tools."""
    tools = [multiply]
    llm_with_tools = llm().bind_tools(tools)
    # ✅ Use dot notation with Pydantic BaseModel
    return {"messages": [llm_with_tools.invoke(state.messages)]}

def tool_execution_node(state: State):
    """Execute tools based on the LLM's tool calls."""
    tool_node = ToolNode([multiply])
    # Convert state to dict for ToolNode
    state_dict = {
        "messages": state.messages,
        "tool_input": state.tool_input,
        "tool_output": state.tool_output
    }
    return tool_node.invoke(state_dict)

def should_continue(state: State):
    """Determine if we should continue to tools or end."""
    messages = state.messages  # ✅ Use dot notation
    last_message = messages[-1]

    # Check if the last message has tool calls
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    else:
        return END

# Build the graph
builder = StateGraph(State)
builder.add_node("node1", node1)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", tool_execution_node)

builder.set_entry_point("node1")
builder.add_edge("node1", "tool_calling_llm")

# Add conditional edge to handle tool calling
builder.add_conditional_edges(
    "tool_calling_llm",
    should_continue,
    {
        "tools": "tools",
        END: END
    }
)

# After tools, go back to LLM for final response
builder.add_edge("tools", "tool_calling_llm")

graph = builder.compile()

# LangGraph Return Type
# Input: Can be Pydantic object or dict
# Output: Always returns a dictionary
# Reason: Internal processing converts everything to dicts


result_dict = graph.invoke({
    "messages": [SystemMessage(content="You are a mathematician"),HumanMessage(content="What is 5 times 6?")],
    "tool_input": "",
    "tool_output": ""
})

print("Final result:")
print(result_dict)

for m in result_dict["messages"]:
    m.pretty_print()

print("\n" + "="*50)
pprint(result_dict["messages"][-1].content)

# In TypedDict

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from typing import Annotated, TypedDict  # ✅ Import TypedDict instead of BaseModel
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

import os, getpass
from pprint import pprint

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

def llm():
    return ChatOpenAI(model="gpt-4o")

# ✅ Use TypedDict instead of Pydantic BaseModel
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    tool_input: str
    tool_output: str

@tool
def multiply(x: int, y: int) -> int:
    """Multiply two integers.

    Args:
        x: First integer
        y: Second integer
    """
    return x * y

def node1(state: State):
    """Initialize conversation if no messages exist."""
    # Check if messages already exist
    return {
            "messages": [HumanMessage(content="What is 4 times 4")],
            "tool_input": "",
            "tool_output": "",
    }

def tool_calling_llm(state: State):
    """LLM node that can call tools."""
    tools = [multiply]
    llm_with_tools = llm().bind_tools(tools)
    # ✅ Use dictionary notation with TypedDict
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

def tool_execution_node(state: State):
    """Execute tools based on the LLM's tool calls."""
    tool_node = ToolNode([multiply])
    # No need to convert state - it's already a dict
    return tool_node.invoke(state)

def should_continue(state: State):
    """Determine if we should continue to tools or end."""
    messages = state["messages"]  # ✅ Use dictionary notation
    last_message = messages[-1]

    # Check if the last message has tool calls
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    else:
        return END

# Build the graph
builder = StateGraph(State)
builder.add_node("node1", node1)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", tool_execution_node)

builder.set_entry_point("node1")
builder.add_edge("node1", "tool_calling_llm")

# Add conditional edge to handle tool calling
builder.add_conditional_edges(
    "tool_calling_llm",
    should_continue,
    {
        "tools": "tools",
        END: END
    }
)

# After tools, go back to LLM for final response
builder.add_edge("tools", "tool_calling_llm")

graph = builder.compile()

result_dict = graph.invoke({
    "messages": [SystemMessage(content="You are a mathematician"), HumanMessage(content="What is 5 times 6?")],
    "tool_input": "",
    "tool_output": ""
})

print("Final result:")
print(result_dict)

for m in result_dict["messages"]:
    m.pretty_print()

print("\n" + "="*50)
pprint(result_dict["messages"][-1].content)

# Additional example showing TypedDict benefits
print("\n" + "="*50)
print("=== TYPEDDICT BENEFITS ===")

# ✅ Simple dictionary creation (no class instantiation needed)
simple_state = {
    "messages": [
        SystemMessage(content="You are a helpful calculator."),
        HumanMessage(content="What is 10 times 3?")
    ],
    "tool_input": "",
    "tool_output": ""
}

result_simple = graph.invoke(simple_state)
print("Simple dictionary invocation works perfectly!")
print(f"Answer: {result_simple['messages'][-1].content}")

# ✅ No need for special Config or Field imports
# ✅ Consistent dictionary access throughout
# ✅ Better performance (no validation overhead)

In [37]:
from pydantic import BaseModel


class state(BaseModel):
    messages: Annotated[List[BaseMessage], add_messages] = Field(default_factory=list)
    tool_input: str = Field(default="")
    tool_output: str = Field(default="")



def Node1(state: state):
    """Initialize conversation if no messages exist."""
    # Check if messages already exist
    return state.model_copy(update={
        "messages": [HumanMessage(content="What is 4 times 4")],
    })




<function node1 at 0x10e5e7a60>
